<a href="https://colab.research.google.com/github/AlinaTkachenko/knowledge_graph_embedding/blob/main/MySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 40.1 MB/s eta 0:00:00


In [ ]:
!apt-get update
!apt-get install -y mysql-server

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mysql-server is already the newest version (8.0.36-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.


In [ ]:
!service mysql start

 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [ ]:
!mysql_secure_installation


Securing the MySQL server deployment.

Connecting to MySQL using a blank password.

VALIDATE PASSWORD COMPONENT can be used to test passwords
and improve security. It checks the strength of password
and allows the users to set only those passwords which are
secure enough. Would you like to setup VALIDATE PASSWORD component?

Press y|Y for Yes, any other key for No: y

There are three levels of password validation policy:

LOW    Length >= 8
MEDIUM Length >= 8, numeric, mixed case, and special characters
STRONG Length >= 8, numeric, mixed case, special characters and dictionary                  file

Please enter 0 = LOW, 1 = MEDIUM and 2 = STRONG: 0

Skipping password set for root as authentication with auth_socket is used by default.
If you would like to use password authentication instead, this can be done with the "ALTER_USER" command.
See https://dev.mysql.com/doc/refman/8.0/en/alter-user.html#alter-user-password-management for more information.

By default, a MySQL installation h

In [ ]:
!mysql -e "CREATE DATABASE test_db;"
!mysql -e "CREATE USER 'test_user'@'localhost' IDENTIFIED BY 'test_pass';"
!mysql -e "GRANT ALL PRIVILEGES ON test_db.* TO 'test_user'@'localhost';"
!mysql -e "FLUSH PRIVILEGES;"


ERROR 1396 (HY000) at line 1: Operation CREATE USER failed for 'test_user'@'localhost'


In [ ]:
# Allow local infile and set secure_file_priv in MySQL configuration
!sed -i '/\[mysqld\]/a local_infile=1\nsecure_file_priv=""' /etc/mysql/mysql.conf.d/mysqld.cnf

In [ ]:
!service mysql restart

 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [ ]:
import pandas as pd
import json
import re

df = pd.read_csv('/content/sample_data/keywords.csv')

# Function to validate and correct JSON strings
def validate_and_correct_json(data):
    if pd.isna(data):
        return '[]'
    # Replace single quotes with double quotes
    data = data.replace("'", '"')
    # Escape any embedded double quotes within strings
    data = re.sub(r'(?<!\\)"', r'\\"', data)
    # Correctly escape backslashes that are not followed by a valid escape character
    data = re.sub(r'\\(?!["\\/bfnrtu])', r'\\\\', data)
    # Handle specific invalid escape sequences
    data = data.replace('\\xa0', ' ')
    # Use a try-except block to catch invalid JSONs and attempt correction
    try:
        json.loads(data)
    except ValueError as e:
        # Additional handling for common issues
        data = data.replace('\\"', '"')  # Unescape quotes for JSON parsing
        data = re.sub(r'\\u([0-9A-Fa-f]{4})', lambda m: chr(int(m.group(1), 16)), data)  # Handle Unicode escapes
        try:
            json.loads(data)
        except ValueError as e:
            print(f"Still Invalid JSON: {data}\nError: {e}")
            return '[]'  # Return an empty JSON array if parsing fails
    return data

# Apply the function to the 'keywords' column
df['keywords'] = df['keywords'].apply(validate_and_correct_json)



# Save the preprocessed DataFrame back to CSV
preprocessed_filename = '/content/sample_data/preprocesed_keywords.csv'
df.to_csv(preprocessed_filename, index=False)


In [ ]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="test_user",
    password="test_pass",
    database="test_db",
    allow_local_infile=True  # Enable local infile
)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS keywords (
    id INT,
    keyword JSON
)
""")

conn.commit()

CSV file contains JSON data that uses single quotes (') instead of double quotes ("). MySQL expects JSON data to use double quotes for property names and string values.

```
# This is formatted as code
```



In [ ]:
cursor.execute("""
LOAD DATA LOCAL INFILE '/content/sample_data/preprocesed_keywords.csv'
INTO TABLE keywords
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES
(id, keyword)
""")

conn.commit()


In [ ]:
# Select and print the results
cursor.execute("SELECT * FROM keywords WHERE JSON_LENGTH(keyword) <> 0;")
rows = cursor.fetchall()

# Print the results
for row in rows:
    print(row)

Streaming output truncated to the last 5000 lines.
(20980, '[{"id": 570, "name": "rape"}, {"id": 572, "name": "sex"}, {"id": 2150, "name": "cocaine"}, {"id": 2671, "name": "heroin"}, {"id": 3183, "name": "bisexuality"}, {"id": 4470, "name": "punk"}, {"id": 9951, "name": "alien"}, {"id": 11402, "name": "androgyny"}, {"id": 14964, "name": "drug"}, {"id": 15479, "name": "fashion"}, {"id": 155490, "name": "disco"}]')
(16407, '[{"id": 478, "name": "china"}, {"id": 779, "name": "martial arts"}, {"id": 11836, "name": "contest"}]')
(41913, '[{"id": 128, "name": "love triangle"}, {"id": 237, "name": "gay"}, {"id": 1862, "name": "coming out"}, {"id": 158718, "name": "lgbt"}]')
(29450, '[{"id": 1415, "name": "small town"}, {"id": 3739, "name": "funeral"}, {"id": 7492, "name": "serbia"}, {"id": 10391, "name": "mafia"}, {"id": 158741, "name": "grave digger"}, {"id": 166512, "name": "graveyard"}, {"id": 214548, "name": "1930s"}]')
(27380, '[{"id": 407, "name": "dictator"}, {"id": 13065, "name": "sol

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

